In [1]:
import pickle
import numpy as np
import argparse
import os
import json
from os import path, listdir
from unidiff import PatchSet, PatchedFile
import warnings
import collections
import random

from commitgen.data import RawDataset, extract_commits, parse_commits
from commitgen.diff import AddRemExtractor, PerFileExtractor, get_added_lines, get_removed_lines
from commitgen.code import CodeChunkTokenizer, CodeLinesTokenizer
from commitgen.nlp import TreebankTokenizer
from pygments.token import Comment, String, Whitespace, Text

In [3]:
work_dir = os.environ['WORK_DIR']
commits_path = 'guava_commits'
commits_path = os.path.join(work_dir, commits_path)
language = 'java'
lexer = CodeLinesTokenizer(language=language)
tokenizer = TreebankTokenizer()
marker = "NEW_FILE"
code_extractor = AddRemExtractor(marker=marker)
parse_filters = []
extract_filters = []
ignore_list = [Comment, String, Whitespace, Text]

In [6]:
tmp_path = os.path.join(commits_path,'json','ff2b5c0897d02d726d803b8103d12ad823c5c4b9.json')

In [8]:
x = open(tmp_path,'r')
x = json.load(x)

In [15]:
x

{'author': {'avatar_url': 'https://avatars.githubusercontent.com/u/1703908?v=3',
  'events_url': 'https://api.github.com/users/cpovirk/events{/privacy}',
  'followers_url': 'https://api.github.com/users/cpovirk/followers',
  'following_url': 'https://api.github.com/users/cpovirk/following{/other_user}',
  'gists_url': 'https://api.github.com/users/cpovirk/gists{/gist_id}',
  'gravatar_id': '',
  'html_url': 'https://github.com/cpovirk',
  'id': 1703908,
  'login': 'cpovirk',
  'organizations_url': 'https://api.github.com/users/cpovirk/orgs',
  'received_events_url': 'https://api.github.com/users/cpovirk/received_events',
  'repos_url': 'https://api.github.com/users/cpovirk/repos',
  'site_admin': False,
  'starred_url': 'https://api.github.com/users/cpovirk/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/cpovirk/subscriptions',
  'type': 'User',
  'url': 'https://api.github.com/users/cpovirk'},
 'comments_url': 'https://api.github.com/repos/google/guava/co

In [3]:
raw_dataset = RawDataset(commits_path)
commits = extract_commits(raw_dataset, code_extractor, filters=extract_filters)
print("Extracted " + str(len(commits)) + " commits")

Extracted 3821 commits


In [4]:
parsed_commits = parse_commits(commits, tokenizer, lexer,
                               filters=parse_filters,
                               ignore_types=ignore_list,
                               marker=marker)

In [6]:
parsed_commits[0]

ParsedCommit(id=0, code='NEW_FILE\n  \n  @Override public final V setValue(V value){\n    throw new UnsupportedOperationException();\n  }\n  \nNEW_FILE', nl_tokens=['Make', 'ImmutableEntry.setValue', '(', ')', 'final.', 'Makes', 'it', 'clearer', 'that', 'ImmutableEntry', 'is', ',', 'in', 'fact', ',', 'immutable.', 'Revision', 'created', 'by', 'MOE', 'tool', 'push_codebase.', 'MOE_MIGRATION=180040', 'git-svn-id', ':', 'https', ':', '//guava-libraries.googlecode.com/svn/trunk', '@', '91', '8138a162-5c33-11de-8abc-d1c337b90d21'], code_tokens=['NEW_FILE', 'NEW_FILE', '@Override', 'public', 'final', 'V', 'setValue', '(', 'V', 'value', ')', '{', 'throw', 'new', 'UnsupportedOperationException', '(', ')', ';', '}'])